By Shirley Yu and Angela Zhang

In [133]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import PolynomialFeatures
import statsmodels.formula.api as smf
from sklearn.cluster import KMeans
import statsmodels.api as sm

In [124]:
one_hot = pd.read_csv("one_hot_responses.csv")
one_hot = one_hot.drop(columns=["Unnamed: 0"])

## PCA to Reduce Dimensionality

In [137]:
# top n pca
n = 50
predictors = [one_hot.columns[i] for i in range(len(one_hot.columns))]
X = one_hot[predictors].values.T

#don't need to standardize since they're dummy variables? idk I asked on piazza
#but when standardizing, if we try to have more than 30 PCs it causes a LinAlgError: Singular matrix error
#n_i, n_j = X.shape
#stds = np.std(X, axis=1)
#stds = np.repeat(np.reshape(stds, (-1,1)), n_j, axis=1)
#S = X/stds

pca = PCA(n_components=n)
pcs = pca.fit_transform(X.T)
df_pcs = pd.DataFrame(pcs, columns=['PC'+str(i+1) for i in range(n)])

# predict whether the person was gay based on their survey responses
df_pcs['y'] = one_hot['q0001'].values

## Logistic Regression for Predicting Masculinity Using Various Numbers of Top PCs

In [63]:
def accuracy(predictions, true):
    correct = (predictions == true)
    acc = sum(correct) / len(correct)
    return acc

In [126]:
# 10 fold cross validation, logistic regression
df_pcs = df_pcs.fillna(0)
mean_mses = []
kf = KFold(n_splits=10, shuffle=True)
pc_numbers = [i+1 for i in range(n)]
# linear models using top k=1,2,3,...,20
for num_pc in pc_numbers:
    # do 10-fold cross validation
    train_mses = []
    test_mses = []
    train_accs = []
    test_accs = []
    for train_idx, test_idx in kf.split(df_pcs):
        df_train = df_pcs.iloc[train_idx, :]
        df_test = df_pcs.iloc[test_idx, :]
        
        # generate formula
        formula = 'y ~ 1'
        for num in range(1, num_pc+1):
            formula += (' + PC%d' % num)
            
        # fit model with df_train
        res = smf.logit(formula=formula, data=df_train).fit(disp=0)

        # MSE of training set
        train_pred = 1*(res.predict(df_train) > 0.5)
        mse_train = np.mean((train_pred - df_train.y)**2)

        # MSE of test set
        test_pred = 1*(res.predict(df_test) > 0.5)
        mse_test = np.mean((test_pred - df_test.y)**2)
        
        acc_train = accuracy(train_pred, df_train.y)
        acc_test = accuracy(test_pred, df_test.y)

        # store results
        train_mses.append(mse_train)
        test_mses.append(mse_test)
        train_accs.append(acc_train)
        test_accs.append(acc_test)

    # report average training MSE and testing MSE
    mean_train_mse = np.mean(train_mses)
    mean_test_mse = np.mean(test_mses)
    mean_train_acc = np.mean(train_accs)
    mean_test_acc = np.mean(test_accs)
    
    mean_mses.append({'num_pc': num_pc,
                      'formula': formula,
                    'mean_train_mse': mean_train_mse,
                     'mean_test_mse': mean_test_mse,
                     'mean_train_acc': mean_train_acc,
                     'mean_test_acc': mean_test_acc})
    
    
df_res = pd.DataFrame(mean_mses)
df_res

formula  mean_test_acc  \
0                                         y ~ 1 + PC1       0.971532   
1                                   y ~ 1 + PC1 + PC2       0.971517   
2                             y ~ 1 + PC1 + PC2 + PC3       0.971540   
3                       y ~ 1 + PC1 + PC2 + PC3 + PC4       0.971509   
4                 y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5       0.971513   
5           y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6       0.971501   
6     y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7       0.970915   
7   y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.970903   
8   y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.970873   
9   y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.972126   
10  y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.970286   
11  y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.970899   
12  y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.970911   
13  y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.969665   
14  y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.968407   
15  y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.969044   
16  y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.970888   
17  y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.969669   
18  y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.969665   
19  y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.967196   
20  y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.969032   
21  y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.966536   
22  y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.965332   
23  y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.968400   
24  y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.968438   
25  y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.967184   
26  y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.967180   
27  y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.969051   
28  y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.967157   
29  y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.968415   
30  y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.967786   
31  y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.968423   
32  y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.967157   
33  y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.966559   
34  y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.969661   
35  y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.969017   
36  y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.965942   
37  y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.967794   
38  y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.965965   
39  y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.966582   
40  y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.967196   
41  y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.966544   
42  y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.963465   
43  y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.964707   
44  y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.965313   
45  y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.964704   
46  y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.965317   
47  y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.965317   
48  y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.964083   
49  y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...       0.965332   

    mean_test_mse  mean_train_acc  mean_train_mse  num_pc  
0        0.028468        0.971517        0.028483       1  
1        0.028483        0.971517        0.028483       2  
2        0.028460        0.971517        0.028483       3  
3        0.028491        0.971517        0.028483       4  
4        0.028487        0.971517        0.028483       5  
5        0.02

### Plotting MSE

In [128]:
fig, ax = plt.subplots()
ax.plot(df_res.num_pc, df_res.mean_train_mse, label='training set')
ax.plot(df_res.num_pc, df_res.mean_test_mse, label='test set')
ax.legend()
#ax.set_xticks(df_res.num_pc.values)
ax.set_xlabel('number of top PC used')
ax.set_ylabel('Mean squared error')
plt.title("MSE for Numbers of Top PC Used")
plt.show()
best_model = df_res.loc[df_res.mean_test_mse.argmin()]
best_model

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  # Remove the CWD from sys.path while we load stuff.


formula           y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...
mean_test_acc                                              0.972126
mean_test_mse                                             0.0278736
mean_train_acc                                             0.972824
mean_train_mse                                            0.0271759
num_pc                                                           10
Name: 9, dtype: object

### Plotting Accuracy

In [95]:
fig, ax = plt.subplots()
ax.plot(df_res.num_pc, df_res.mean_train_acc, label='training set')
ax.plot(df_res.num_pc, df_res.mean_test_acc, label='test set')
ax.legend()
#ax.set_xticks(df_res.num_pc.values)
ax.set_xlabel('number of top PC used')
ax.set_ylabel('Accuracy')
plt.title("Accuracy for Numbers of Top PC Used")
plt.show()
best_model = df_res.loc[df_res.mean_test_acc.argmax()]
best_model

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
  # Remove the CWD from sys.path while we load stuff.


formula           y ~ 1 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC...
mean_test_acc                                              0.972755
mean_test_mse                                             0.0272446
mean_train_acc                                             0.972136
mean_train_mse                                            0.0278638
num_pc                                                           10
Name: 9, dtype: object

### Explained Variance and Most Imporant Feature for each PC

In [122]:
print(pca.explained_variance_ratio_) 

[0.13063776 0.05241703 0.0446671  0.02854428 0.02477324 0.02168846
 0.01967302 0.01873948 0.01759851 0.01754495 0.01530759 0.01482918
 0.01430431 0.01343961 0.01316621 0.01263012 0.0120022  0.01189397
 0.01155233 0.01141751 0.01114379 0.01103642 0.01072142 0.01055201
 0.01033964 0.00989237 0.0098532  0.0096393  0.00932993 0.00904221
 0.00880386 0.00869686 0.00850667 0.00846557 0.00831656 0.00816874
 0.00805011 0.00779247 0.00777166 0.00756868 0.00740771 0.00727339
 0.007166   0.00700152 0.00689503 0.00666616 0.00648185 0.0063594
 0.00619028 0.00599489]


In [118]:
feature_importance = pd.DataFrame(pca.components_,columns=one_hot.columns,index = ['PC'+str(i+1) for i in range(n)])

In [120]:
feature_importance.idxmax(axis=1)

PC1                            q0009_Not employed-retired
PC2                                q0025_0003_No children
PC3                                q0025_0003_No children
PC4                               q0025_0003_Not selected
PC5                               q0004_0002_Not selected
PC6                               q0011_0004_Not selected
PC7     q0020_0001_Read their physical body language t...
PC8                               q0020_0005_Not selected
PC9                                             q0005_Yes
PC10                              q0020_0004_Not selected
PC11                                             q0005_No
PC12              q0020_0004_Every situation is different
PC13                                    q0007_0004_Rarely
PC14                              q0020_0004_Not selected
PC15                                             q0017_No
PC16                              q0020_0004_Not selected
PC17                    q0004_0006_Other (please specify)
PC18          